## Hangman - A Word-guessing Game ##
https://en.wikipedia.org/wiki/Hangman_(game)

Code by Zihang Pan

In [1]:
import numpy as np
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import operator
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [2]:
## Train set dictionary (full dictionary)
text_file = open("words_250000_train.txt", "r")
full_dictionary = text_file.read().splitlines()
text_file.close()

In [3]:
## Test set dictionary, NOT USED IN TRAINING!!!
text_file = open("words_alpha.txt", "r")
external_dictionary = text_file.read().splitlines()
text_file.close()
test_dictionary = np.setdiff1d(external_dictionary, full_dictionary)

In [4]:
def sub_count(full_dictionary, sub_length):
    """Function to count the occurrences of all substrings of certain length (given by sub_length) in the full dictionary.
    Return a list of tuples containing substrings and their occurrences, sorted by occurrences.
    Example: Substrings of length 4: [('tion', 5818), ('ness', 5531), ('atio', 3995), ('ical', 3296), ('able', 2986), ...]
    """
    substring_dict = dict()
    for word in full_dictionary:
        if len(word) < sub_length:
            continue
        for i in range(len(word) - sub_length + 1):
            substring = word[i:i+sub_length]
            if substring in substring_dict.keys():
                substring_dict[substring] += 1
            else:
                substring_dict[substring] = 1
   
    substring_dict = sorted(substring_dict.items(), key = operator.itemgetter(1), reverse = True)
    
    return substring_dict
        
    

In [5]:
def pref_count(full_dictionary, sub_length):
    """Function to count prefix substrings, in a same logic.
    NOT implemented, since using substrings for 2,3,4,5 letters already reached over 50%.
    """
    substring_dict = dict()
    for word in full_dictionary:
        if len(word) < sub_length:
            continue
        i = 0
        substring = word[i:i+sub_length]
        if substring in substring_dict.keys():
            substring_dict[substring] += 1
        else:
            substring_dict[substring] = 1

    substring_dict = sorted(substring_dict.items(), key = operator.itemgetter(1), reverse = True)
    
    return substring_dict

In [6]:
def match_substring(word, i, substring_dictionary, sub_len, letter_dictionary, guessed_letters):
    """Function to guess the i-th letter in the word.
    The word is masked by ".", for example ".ord". 
    The function will match the substring of ".o" to length-2 substrings in our pre-generated substring dictionary,
    and record the occurrences of the letters at the position of ".". 
    guessed_letters is the list of letters that the player already used, these letters shouldn't be going to masked position. 
    Return a dictionary containing the possibility of each letter showing up at position i. 
    (Possibilities are weighted by total occurrences.)
    """
    skip = False
    for position in range(sub_len):
        if i-position >= 0 and i+sub_len-position <= len(word):
            sub_match = word[i-position:i+sub_len-position]
            revealed = sub_len - sub_match.count('.')
            for sub_pattern in substring_dictionary:
                if re.match(sub_match, sub_pattern[0]):
                    for guessed in guessed_letters:
                        if sub_pattern[0].count(guessed) > sub_match.count(guessed):
                            skip = True
                            break
                        else:
                            skip = False
                    if skip == False:
                        letter_dictionary[sub_pattern[0][position]] += (revealed+1)**2 * sub_pattern[1]
    
    total = sum(letter_dictionary.values(), 0)
    if total != 0:
        letter_dictionary = {k: v / total for k, v in letter_dictionary.items()}
    
    return letter_dictionary

In [7]:
## Guess function take in pre-generated substring dictionaries for length 2,3,4,5 and guessed letters
## Guess the letter with max probablity to appear on one spot.

def guess(word, two_sub_count, three_sub_count, four_sub_count, five_sub_count, guessed_letters):
    """Perform one try guessing the letter that are most likely to appear, given current word mask. 
    Looping through each masked position, apply the above match_substring function to match substrings of length 2,3,4,5.
    Keep track of the letter with max possibilty (given by those substring matching) at each position.
    Filter off guessed letters. Select the most possible letter to represent this position. 
    After looping through every masked position, select the most possible letter in all position as the guess for this try. 
    """
    all_letters = 'abcdefghijklmnopqrstuvwxyz'
    letter_dict = dict() 
    max_prob_at_i = []
    
    ## Loop through each position in the word
    
    for i in range(len(word)):
        skip = False
        for l in all_letters:
            letter_dict[l] = 0
        
        two_letter_dict = letter_dict.copy()
        three_letter_dict = letter_dict.copy()
        four_letter_dict = letter_dict.copy()
        five_letter_dict = letter_dict.copy()

        if word[i] != '.':
            continue
                
        ## Get letter possibilies at this position for matching of different length
        
        two_letter_dict = match_substring(word, i, two_sub_count, 2, two_letter_dict, guessed_letters)
        three_letter_dict = match_substring(word, i, three_sub_count, 3, three_letter_dict, guessed_letters)
        four_letter_dict = match_substring(word, i, four_sub_count, 4, four_letter_dict, guessed_letters)
        five_letter_dict = match_substring(word, i, five_sub_count, 5, five_letter_dict, guessed_letters)
        
        ## For every letter at this position, take the max posibility among the four dictionaries. Put them into one alphabet.     
        
        for l in all_letters:
            letter_dict[l] = max(two_letter_dict[l],three_letter_dict[l],four_letter_dict[l], five_letter_dict[l])
        
        ## Take the unguessed letter with max posibility appearance as the best letter for this position
        
        i_letter_dict = sorted(letter_dict.items(), key = operator.itemgetter(1), reverse = True)
        
        for letter in i_letter_dict:
            if letter[0] not in guessed_letters:
                max_prob_at_i.append(letter)
                break
        
    ## After looping through the word, we have a best letter for every position. Guess the letter with max posibility.
    
    max_letter = ''
    max_prob = 0
    for j in max_prob_at_i:
        if j[1] > max_prob:
            max_prob = j[1]
            max_letter = j[0]
    
    return(max_letter)
        
                    
            
    

In [8]:
def gen_secret(secret_dictionary):
    """Randomly generate a secret word from provided dictionary
    """
    id = random.randint(0,len(secret_dictionary))
    secret = secret_dictionary[id]
    return secret

In [9]:
def start_game(full_dictionary, secret_dictionary, total_tries, two_sub_count, three_sub_count, four_sub_count, five_sub_count):
    """Start a game: the game will automatically generate a secret and try guessing one letter at a time, until revealing the correct word or reaching total_tries.
    Return 1 if successfully guessed the word, 0 if failed. 
    """
    # The secret_dictionary is ONLY used for generating the secret
    secret = gen_secret(secret_dictionary)
    
    print("Secret Generated !")
    # print("_________________________")
    
    temp_two_sub_count = two_sub_count.copy()
    temp_three_sub_count = three_sub_count.copy()
    temp_four_sub_count = four_sub_count.copy()
    temp_five_sub_count = five_sub_count.copy()
    
    guessed_letters = []
    
    # create the mask for secret
    length = len(secret)
    mask = ""
    for i in range(length):
        mask = mask + "."
    
    n_letters_left = length
    n_tries_left = total_tries
    current_mask = mask

    while n_letters_left > 0:
        # print("")
        # print("Current:", current_mask)
        # print("tries left:", n_tries_left)
        if n_tries_left == 0:
            print("Failed - Tries exhausted")
            print("Final:", current_mask)
            print("Answer:", secret)
            return(0)
            
        guess_letter = guess(current_mask, temp_two_sub_count, temp_three_sub_count, temp_four_sub_count, temp_five_sub_count, guessed_letters)
        # print("Guess:", guess_letter)
        guessed_letters.append(guess_letter)
        # print("Guessed_letters:", guessed_letters)
       
        
        if guess_letter in secret:
            # print("Correct guess!")
            count_letters = 0
            for i in range(len(secret)):
                if secret[i] == guess_letter:
                    current_mask_list = list(current_mask)
                    current_mask_list[i] = guess_letter
                    current_mask = "".join(current_mask_list)
                    count_letters += 1
            n_letters_left -= count_letters
            
            
        else:
            # print("Wrong guess!")
            n_tries_left -= 1

            
    print("Success!")
    print("Answer:", secret)
    return(1)        
    
    
    

In [10]:
## Pre-generate dictionaries before starting a game.
## Note that there are too many four-letter-substrings and five-letter-substrings, so I just throwed away those with small sample sizes.

two_sub_count = sub_count(full_dictionary, 2)
three_sub_count = sub_count(full_dictionary, 3)
four_sub_count = sub_count(full_dictionary, 4)
five_sub_count = sub_count(full_dictionary, 5)
four_sub_count = [i for i in four_sub_count if i[1] > 3]
five_sub_count = [i for i in five_sub_count if i[1] > 3]

In [11]:
## Now try one game!
start_game(full_dictionary, test_dictionary, 6, two_sub_count, three_sub_count, four_sub_count, five_sub_count)

Secret Generated !
Failed - Tries exhausted
Final: te..ira.
Answer: tezkirah


0

# Simulation

Simulations on the training set. Should comment off unnecessary 'print' in 'start_game' before running large number of games.

In [12]:
n = 50
test_result = np.zeros(n)
two_sub_count = sub_count(full_dictionary, 2)
three_sub_count = sub_count(full_dictionary, 3)
four_sub_count = sub_count(full_dictionary, 4)
five_sub_count = sub_count(full_dictionary, 5)
four_sub_count = [i for i in four_sub_count if i[1] > 3]
five_sub_count = [i for i in five_sub_count if i[1] > 3]
for i in range(n):
    print("GAME:",i)
    test_result[i] = start_game(full_dictionary, full_dictionary, 6, two_sub_count, three_sub_count, four_sub_count, five_sub_count)
    print("\n")

GAME: 0
Secret Generated !
Failed - Tries exhausted
Final: .a.de.es
Answer: caudexes


GAME: 1
Secret Generated !
Failed - Tries exhausted
Final: .u..i.i.t.
Answer: publicists


GAME: 2
Secret Generated !
Failed - Tries exhausted
Final: .ar.han
Answer: pardhan


GAME: 3
Secret Generated !
Failed - Tries exhausted
Final: anti.a.
Answer: antigay


GAME: 4
Secret Generated !
Success!
Answer: achier


GAME: 5
Secret Generated !
Success!
Answer: flatulences


GAME: 6
Secret Generated !
Success!
Answer: stringbinding


GAME: 7
Secret Generated !
Success!
Answer: oversecurity


GAME: 8
Secret Generated !
Success!
Answer: subbrachial


GAME: 9
Secret Generated !
Failed - Tries exhausted
Final: .us.
Answer: busy


GAME: 10
Secret Generated !
Success!
Answer: athecata


GAME: 11
Secret Generated !
Failed - Tries exhausted
Final: ..
Answer: fb


GAME: 12
Secret Generated !
Failed - Tries exhausted
Final: .a..a.
Answer: gallas


GAME: 13
Secret Generated !
Failed - Tries exhausted
Final: .riia
Ans

In [13]:
success_rate = sum(test_result)/n
print("success:", sum(test_result))
print("rate:", success_rate)

success: 27.0
rate: 0.54


# Test Set

Simulations on the test set, which has completely new words without knowledge in 'full dictionary'.

In [14]:
n = 50
test_result = np.zeros(n)
two_sub_count = sub_count(full_dictionary, 2)
three_sub_count = sub_count(full_dictionary, 3)
four_sub_count = sub_count(full_dictionary, 4)
five_sub_count = sub_count(full_dictionary, 5)
four_sub_count = [i for i in four_sub_count if i[1] > 3]
five_sub_count = [i for i in five_sub_count if i[1] > 3]
for i in range(n):
    print("GAME:",i)
    test_result[i] = start_game(full_dictionary, test_dictionary, 6,  two_sub_count, three_sub_count, four_sub_count, five_sub_count)

GAME: 0
Secret Generated !
Success!
Answer: microsporosis
GAME: 1
Secret Generated !
Success!
Answer: rancor
GAME: 2
Secret Generated !
Success!
Answer: parachromatosis
GAME: 3
Secret Generated !
Success!
Answer: nitrosamine
GAME: 4
Secret Generated !
Success!
Answer: obstinative
GAME: 5
Secret Generated !
Failed - Tries exhausted
Final: e.eri.h
Answer: everich
GAME: 6
Secret Generated !
Failed - Tries exhausted
Final: .o.io.ia
Answer: copiopia
GAME: 7
Secret Generated !
Success!
Answer: rodwood
GAME: 8
Secret Generated !
Success!
Answer: overidentified
GAME: 9
Secret Generated !
Failed - Tries exhausted
Final: r..estrian
Answer: rupestrian
GAME: 10
Secret Generated !
Success!
Answer: enigma
GAME: 11
Secret Generated !
Failed - Tries exhausted
Final: slea.iest
Answer: sleaziest
GAME: 12
Secret Generated !
Success!
Answer: inker
GAME: 13
Secret Generated !
Success!
Answer: somatotyper
GAME: 14
Secret Generated !
Success!
Answer: matin
GAME: 15
Secret Generated !
Success!
Answer: pronate

In [15]:
success_rate = sum(test_result)/n
print("success:", sum(test_result))
print("rate:", success_rate)

success: 36.0
rate: 0.72
